# Imports

In [762]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import html5lib
import lxml
import re
from datetime import datetime
import time
import itertools
import random
import numpy as np
from io import BytesIO
from math import radians, cos, sin, asin, sqrt, atan2
import sqlite3 as db


# Setup

In [763]:
# OHSAA Southwest District conferences
domains = ['https://eccsports.com/',
           'https://gmcsports.com/',
           'http://www.swocsports.com/',
           'http://www.swblsports.com/',
           'http://sbaac.com/',
           'http://ggcl.gclsports.com/',
           ]

# Gender URL examples (standings, schedule, statistics)
girls = ['35', '215', '218']
boys = ['30', '137', '162']

# Requests headers
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}


# Download List of Matches

In [764]:
match_ids = []
totals_rows_list = []
for year in range(2007, 2023):
    for gender in [girls, boys]:
        for domain in domains[:]:
            page = requests.get(
                f'{domain}schedule.aspx?satc={gender[1]}&cmp=1&year={year}', headers=headers)
            soup = BeautifulSoup(page.content, 'html.parser')
            title = soup.find('title').string
            match_links = soup.select('a[id*=ContentPlaceHolder1]')
            for link in match_links:
                href = link['href']
                matches = re.findall(r'id=(.*)', href)

                # Add to list of Match IDs
                match_ids.append(matches[0])

                # Add to list of Match IDs with totals from table
                id = int(matches[0])
                total = link.parent.find_previous_sibling().findChild().contents
                date = link.find_previous('th').find_previous('th').contents
                date_parsed = str(datetime.strptime(
                    date[0], '%A, %B %d, %Y').isoformat()[:10])

                row = [id, total, date_parsed,
                       f'{year}-{year+1}', gender[1], domain, title]
                totals_rows_list.append(row)
            time.sleep(1)


In [765]:
df_matches_totals = pd.DataFrame(totals_rows_list, columns=[
                                 'match_id', 'total', 'date', 'year', 'gender', 'domain', 'title'])
df_matches_totals['total'] = df_matches_totals['total'].str[0]
df_matches_totals.to_csv('matches_sw_district.csv')
df_matches_totals


,match_id,total,date,year,gender,domain,title
0,22219,"Loveland 1602, Hamilton 1313",2008-11-24,2008-2009,215,https://gmcsports.com/,Greater Miami Conference
1,22220,"Middletown 2206, Sycamore 1976",2008-11-24,2008-2009,215,https://gmcsports.com/,Greater Miami Conference
2,22222,"Princeton 2318, Colerain 1895",2008-11-25,2008-2009,215,https://gmcsports.com/,Greater Miami Conference
3,22221,"Mason 1827, Hamilton 1465",2008-11-25,2008-2009,215,https://gmcsports.com/,Greater Miami Conference
4,23966,"Lakota East 2072, Walnut Hills 1728",2008-12-01,2008-2009,215,https://gmcsports.com/,Greater Miami Conference
...,...,...,...,...,...,...,...
11028,237373,"New Richmond 2741, McNicholas 2502",2022-02-08,2021-2022,137,http://sbaac.com/,Southern Buckeye Athletic and Academic Conference
11029,218533,"Clermont Northeastern 2256, East Clinton 2112",2022-02-08,2021-2022,137,http://sbaac.com/,Southern Buckeye Athletic and Academic Conference
11030,237374,"Georgetown 2489, New Richmond 2337",2022-02-11,2021-2022,137,http://sbaac.com/,Southern Buckeye Athletic and Academic Conference
11031,240020,Felicity-Franklin at Sectional Tournament **sc...,2022-02-15,2021-2022,137,http://sbaac.com/,Southern Buckeye Athletic and Academic Conference


## Filter out matches without scores or outside usual

In [767]:
has_score_filter = df_matches_totals['total'].str.contains('(\d\d\d.*){2}')
df_has_score = df_matches_totals[has_score_filter]
not_actual_match = df_has_score['total'].str.contains(
    '(\d\d\d.+){3,}|( Tournament )|( at )|(lassic)|(,.+){2}')
df_has_score = df_has_score[~not_actual_match]


C:\Users\J\AppData\Local\Temp\ipykernel_12012\3542400068.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  has_score_filter = df_matches_totals['total'].str.contains('(\d\d\d.*){2}')
C:\Users\J\AppData\Local\Temp\ipykernel_12012\3542400068.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  not_actual_match = df_has_score['total'].str.contains(


In [ ]:
# df_has_score.to_csv('matches_sw_district_have_scores.csv')


## Split score totals

In [769]:
df_score_split = df_has_score.copy()
df_score_split['total'] = df_score_split['total'].str.split(',')
df_score_split = df_score_split.explode('total').reset_index(drop=True)
df_score_split['name'] = df_score_split['total'].str[:-5]
df_score_split['score'] = df_score_split['total'].str[-4:]


In [ ]:
# df_score_split.to_csv('matches_sw_district_have_scores_split.csv')


## Download baker games

In [770]:
baker_games = []
for ind in df_matches_totals.sample(frac=1).index:
    page = requests.get(f'{df_matches_totals.domain[ind]}bwMatchStats.aspx?matchid={df_matches_totals.match_id[ind]}', headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    ths = soup.find_all('th', string='BAKER GAMES')
    for i in range(len(ths)):
        games_list = []
        school_th = ths[i].findPrevious('thead').findNext('th').text
        for j in range(6):
            games_list.append(ths[i].parent.findNext('td').find_next_siblings()[j].text)
        baker_row = (df_matches_totals.domain[ind], df_matches_totals.match_id[ind], school_th, games_list)
        baker_games.append(baker_row)
        time.sleep(1)


In [771]:
df_baker_games = pd.DataFrame(baker_games, columns=['domain', 'match_id', 'school', 'games'])
# df_baker_games.to_csv('baker_games.csv')
df_baker_games

,domain,match_id,school,games
0,https://eccsports.com/,104521,Glen Este,"[188, 167, 207, 212, 179, 174]"
1,http://ggcl.gclsports.com/,22764,McNicholas,"[114, 130, 118, 106, , ]"
2,http://ggcl.gclsports.com/,22764,Purcell Marian,"[116, 108, 139, 159, , ]"
3,https://gmcsports.com/,66868,Fairfield,"[140, 160, 153, 149, 126, 174]"
4,http://www.swocsports.com/,239167,Northwest,"[150, 116, 137, 142, , ]"
...,...,...,...,...
14281,http://ggcl.gclsports.com/,57046,Chaminade Julienne,"[169, 155, 145, 100, , ]"
14282,http://sbaac.com/,239023,Western Brown,"[169, 128, 143, , , ]"
14283,http://ggcl.gclsports.com/,58764,Roger Bacon,"[128, 134, 130, 95, , ]"
14284,http://ggcl.gclsports.com/,51538,Mercy,"[118, 197, 198, 178, , ]"


In [772]:
df_baker_games_split = df_baker_games['games'].apply(pd.Series)


In [773]:
df_baker_games_split.rename(columns={i: "game"+str(i+1)
                      for i in range(len(df_baker_games_split.columns))}, inplace=True)
df_baker_games_split = df_baker_games_split.replace('', np.nan, regex=True)
df_baker_games_split = df_baker_games_split.apply(pd.to_numeric, errors='coerce', downcast='integer')
df_baker_games_split = df_baker_games_split.astype('Int16')
df_baker_games_split

,game1,game2,game3,game4,game5,game6
0,188,167,207,212,179,174
1,114,130,118,106,<NA>,<NA>
2,116,108,139,159,<NA>,<NA>
3,140,160,153,149,126,174
4,150,116,137,142,<NA>,<NA>
...,...,...,...,...,...,...
14281,169,155,145,100,<NA>,<NA>
14282,169,128,143,<NA>,<NA>,<NA>
14283,128,134,130,95,<NA>,<NA>
14284,118,197,198,178,<NA>,<NA>


In [774]:
df_baker_games_split['count'] =  df_baker_games_split[df_baker_games_split[:] > 1].count(axis=1)
# Calculate standard deviation, variance, mean, maximum, minimum, range, and total pinfall
df_baker_games_split['std'] = np.std(df_baker_games_split.iloc[:, :6], axis=1).round(0)
df_baker_games_split['var'] = np.var(df_baker_games_split.iloc[:, :6], axis=1).round(0)
df_baker_games_split['avg'] = df_baker_games_split.iloc[:, :6].mean(axis=1).round(0)
df_baker_games_split['max'] = df_baker_games_split.iloc[:, :6].max(axis=1)
df_baker_games_split['min'] = df_baker_games_split.iloc[:, :6].min(axis=1)
df_baker_games_split['rng'] = df_baker_games_split['max'] - df_baker_games_split['min']
df_baker_games_split['tot'] = df_baker_games_split.iloc[:, :6].sum(axis=1)
df_baker_games_split

,game1,game2,game3,game4,game5,game6,count,std,var,avg,max,min,rng,tot
0,188,167,207,212,179,174,6,17.0,276.0,188.0,212.0,167.0,45.0,1127.0
1,114,130,118,106,<NA>,<NA>,4,9.0,75.0,117.0,130.0,106.0,24.0,468.0
2,116,108,139,159,<NA>,<NA>,4,20.0,400.0,130.0,159.0,108.0,51.0,522.0
3,140,160,153,149,126,174,6,15.0,227.0,150.0,174.0,126.0,48.0,902.0
4,150,116,137,142,<NA>,<NA>,4,13.0,158.0,136.0,150.0,116.0,34.0,545.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14281,169,155,145,100,<NA>,<NA>,4,26.0,668.0,142.0,169.0,100.0,69.0,569.0
14282,169,128,143,<NA>,<NA>,<NA>,3,17.0,287.0,147.0,169.0,128.0,41.0,440.0
14283,128,134,130,95,<NA>,<NA>,4,16.0,243.0,122.0,134.0,95.0,39.0,487.0
14284,118,197,198,178,<NA>,<NA>,4,33.0,1063.0,173.0,198.0,118.0,80.0,691.0


In [775]:
df_baker_gamestats_split = pd.concat([df_baker_games, df_baker_games_split], axis=1)
df_baker_gamestats_split = df_baker_gamestats_split.drop('games', axis=1)
df_baker_gamestats_split

,domain,match_id,school,game1,game2,game3,game4,game5,game6,count,std,var,avg,max,min,rng,tot
0,https://eccsports.com/,104521,Glen Este,188,167,207,212,179,174,6,17.0,276.0,188.0,212.0,167.0,45.0,1127.0
1,http://ggcl.gclsports.com/,22764,McNicholas,114,130,118,106,<NA>,<NA>,4,9.0,75.0,117.0,130.0,106.0,24.0,468.0
2,http://ggcl.gclsports.com/,22764,Purcell Marian,116,108,139,159,<NA>,<NA>,4,20.0,400.0,130.0,159.0,108.0,51.0,522.0
3,https://gmcsports.com/,66868,Fairfield,140,160,153,149,126,174,6,15.0,227.0,150.0,174.0,126.0,48.0,902.0
4,http://www.swocsports.com/,239167,Northwest,150,116,137,142,<NA>,<NA>,4,13.0,158.0,136.0,150.0,116.0,34.0,545.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14281,http://ggcl.gclsports.com/,57046,Chaminade Julienne,169,155,145,100,<NA>,<NA>,4,26.0,668.0,142.0,169.0,100.0,69.0,569.0
14282,http://sbaac.com/,239023,Western Brown,169,128,143,<NA>,<NA>,<NA>,3,17.0,287.0,147.0,169.0,128.0,41.0,440.0
14283,http://ggcl.gclsports.com/,58764,Roger Bacon,128,134,130,95,<NA>,<NA>,4,16.0,243.0,122.0,134.0,95.0,39.0,487.0
14284,http://ggcl.gclsports.com/,51538,Mercy,118,197,198,178,<NA>,<NA>,4,33.0,1063.0,173.0,198.0,118.0,80.0,691.0


In [776]:
df_baker_gamestats_split.to_csv('baker_gamestats_split.csv')

# Download List of Schools

In [777]:
schools_rows_list = []
for domain in domains[:]:
    for gender in [girls, boys]:
        for year in range(2007, 2023):
            page = requests.get(
                f'{domain}confstandings.aspx?sat={gender[0]}&cmp=1&year={year}', headers=headers)
            soup = BeautifulSoup(page.content, 'html.parser')
            school_links = soup.select('a[id*=ContentPlaceHolder1]')
            for link in school_links:

                school_name = link.contents

                schools_rows_list.append(school_name)
            time.sleep(5)


In [778]:
df_official_names = pd.DataFrame(schools_rows_list, columns=['Name'])
df_official_names = df_official_names.drop_duplicates().reset_index(drop=True)


In [ ]:
# df_official_names.to_csv('official_names.csv')


# Download Player Stats

In [779]:
school_ids = []
schools_rows_list = []

for year in range(2007, 2023):
    for gender in [girls, boys]:
        for domain in domains[:]:
            page = requests.get(
                f'{domain}bwstatistics.aspx?satc={gender[2]}&year={year}', headers=headers)
            soup = BeautifulSoup(page.content, 'html.parser')
            stats_links = soup.select('a[href*=teamStats]')
            for link in stats_links:
                href = link['href']
                matches = re.findall(r'id=(.*)', href)
                school_ids.append(matches[0])
                id = int(matches[0])
                row = [id, domain]
                schools_rows_list.append(row)
                time.sleep(5)


In [780]:
schools_rows_list.sort()
unique_schools = list(k for k, _ in itertools.groupby(schools_rows_list))
df_unique_schools = pd.DataFrame(
    unique_schools, columns=['SchoolID', 'Domain'])
unique_schools


[[1, 'https://gmcsports.com/'],
 [2, 'https://gmcsports.com/'],
 [3, 'https://gmcsports.com/'],
 [4, 'https://gmcsports.com/'],
 [5, 'https://gmcsports.com/'],
 [6, 'https://gmcsports.com/'],
 [7, 'https://eccsports.com/'],
 [8, 'https://gmcsports.com/'],
 [9, 'https://gmcsports.com/'],
 [10, 'https://gmcsports.com/'],
 [11, 'http://ggcl.gclsports.com/'],
 [12, 'http://ggcl.gclsports.com/'],
 [13, 'http://ggcl.gclsports.com/'],
 [16, 'http://ggcl.gclsports.com/'],
 [18, 'http://ggcl.gclsports.com/'],
 [19, 'http://ggcl.gclsports.com/'],
 [22, 'https://eccsports.com/'],
 [23, 'https://eccsports.com/'],
 [24, 'http://sbaac.com/'],
 [25, 'http://www.swocsports.com/'],
 [26, 'https://eccsports.com/'],
 [27, 'http://www.swocsports.com/'],
 [27, 'https://eccsports.com/'],
 [28, 'https://eccsports.com/'],
 [29, 'https://gmcsports.com/'],
 [31, 'http://www.swocsports.com/'],
 [32, 'http://www.swocsports.com/'],
 [33, 'https://eccsports.com/'],
 [34, 'https://eccsports.com/'],
 [35, 'http://sba

In [ ]:
# df_unique_schools.to_csv('school_ids.csv')


In [781]:
players_rows_list = []
random.shuffle(unique_schools)
for year in range(2007, 2023):
    for gender in [girls, boys]:
        for school_id, domain in unique_schools[:]:
            page = requests.get(
                f'{domain}teamStats.aspx?sat={gender[0]}&cmp=1&year={year}&schoolid={school_id}', headers=headers)
            soup = BeautifulSoup(page.content, 'html.parser')
            player_links = soup.select('a[id*=ContentPlaceHolder1]')
            for link in player_links:
                href = link['href']
                matches = re.findall(r'player=(.*)', href)

                # Add to list of Match IDs with totals from table
                id = int(matches[0])

                row = [id, domain, ]
                players_rows_list.append(row)
            time.sleep(1)
df_player_ids = pd.DataFrame(players_rows_list, columns=[
                             'player_id', 'domain', ])


In [ ]:
# df_player_ids.to_csv('player_ids.csv')


In [782]:
gamestats = []
random.shuffle(players_rows_list)
for p_id, dom in players_rows_list:
    page = requests.get(f'{dom}playerStats.aspx?player={p_id}', headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    title = soup.find('title').string
    schoolH1 = soup.select('h1')
    player_name = soup.select('h2')[0].contents[0].text
    try:
        grade_lvl = soup.select('span[id*=GradeVal]')[0].contents[0].text
    except:
        grade_lvl = ''
    match_list = soup.select('a[id*=GamesRepeat]')
    # soup.select('a[id*=ContentPlaceHolder1]')
    schname = schoolH1[0].contents[0].text
    genname = schoolH1[0].contents[2].text
    if title == 'Girls Greater Catholic League':
        ind = 0
        genname = 'Girls'
    else:
        ind = 1
    schgen = [re.findall(pattern='(\w+.*)', string=schname)
              [0], re.findall(pattern='(\w+)', string=genname)[ind]]
    for i in range(len(match_list)):
        href = match_list[i]['href']
        match_id = re.findall(r'ID=(.*)', href)
        pins = match_list[i].find_previous().find_previous_siblings()[3].text
        location = match_list[i].find_previous().find_previous_siblings()[
            5].text
        opponent = match_list[i].find_previous().find_previous_siblings()[
            6].text
        row = [player_name, p_id, schgen[1], grade_lvl,
               schgen[0], match_id[0], location, opponent, pins, ]
        gamestats.append(row)
    time.sleep(1)


In [783]:
df_gamestats = pd.DataFrame(gamestats, columns=[
                            'player_name', 'player_id', 'gender', 'grade_level', 'school', 'match_id', 'location', 'opponent', 'pins'])
# Count number of games in match
df_gamestats['num'] = df_gamestats['pins'].str.count(',') + 1
df_gamestats


,player_name,player_id,gender,grade_level,school,match_id,location,opponent,pins,num
0,Jennifer Calai,505715,Girls,Junior,Mt. Healthy,130115,Oxford Lanes,Edgewood,68,1
1,Jennifer Calai,505715,Girls,Junior,Mt. Healthy,132603,Eastgate Lanes,Little Miami,"90, 104",2
2,Diamond Johnson,314151,Girls,Senior,Withrow,78080,Stones Lanes,Turpin,86,1
3,Diamond Johnson,314151,Girls,Senior,Withrow,78575,Stone Lanes,Winton Woods,"111, 101",2
4,Diamond Johnson,314151,Girls,Senior,Withrow,78576,Stone Lanes,Loveland,"91, 72",2
...,...,...,...,...,...,...,...,...,...,...
89528,Jaria Gray,96381,Girls,Senior,Chaminade Julienne,30458,Poelking Lanes,Fenwick,"97, 137",2
89529,Jaria Gray,96381,Girls,Senior,Chaminade Julienne,30462,Poelking Lanes,Carroll,137,1
89530,Jaria Gray,96381,Girls,Senior,Chaminade Julienne,30927,Poelking Lanes,Springboro,"156, 165",2
89531,Jaria Gray,96381,Girls,Senior,Chaminade Julienne,34409,Poelking Lanes,Franklin,"119, 130",2


In [ ]:
# df_gamestats.to_csv('gamestats.csv')


In [784]:
# Remove whitespace in pins column
df_gamestats['pins'] = df_gamestats['pins'].replace(' ', '', regex=True)
# Split pins to individual games and rename new columns
df_games_split = pd.DataFrame(df_gamestats['pins'].str.split(',').tolist(), )
df_games_split.rename(columns={i: "game"+str(i+1)
                      for i in range(len(df_games_split.columns))}, inplace=True)
# Convert to int and drop all-NaN rows
df_games_split = df_games_split.apply(pd.to_numeric, downcast='integer')
df_games_split.dropna(how='all', inplace=True)
# Calculate standard deviation, variance, median, mean, maximum, minimum, range, and total pinfall
df_games_split['std'] = np.std(df_games_split, axis=1).round(0)
df_games_split['var'] = np.var(df_games_split, axis=1).round(0)
df_games_split['med'] = np.nanmedian(df_games_split, axis=1).round(0)
df_games_split['avg'] = df_games_split.iloc[:, :-3].mean(axis=1).round(0)
df_games_split['max'] = df_games_split.iloc[:, :-4].max(axis=1)
df_games_split['min'] = df_games_split.iloc[:, :-5].min(axis=1)
df_games_split['rng'] = df_games_split['max'] - df_games_split['min']
df_games_split['tot'] = df_games_split.iloc[:, :-7].sum(axis=1)
# Re-convert to int
df_games_split = df_games_split.astype('Int16')
df_games_split


,game1,game2,game3,game4,game5,game6,std,var,med,avg,max,min,rng,tot
0,68,<NA>,<NA>,<NA>,<NA>,<NA>,0,1156,68,68,68,68,0,68
1,90,104,<NA>,<NA>,<NA>,<NA>,7,1833,97,97,104,90,14,194
2,86,<NA>,<NA>,<NA>,<NA>,<NA>,0,1849,86,86,86,86,0,86
3,111,101,<NA>,<NA>,<NA>,<NA>,5,2284,106,106,111,101,10,212
4,91,72,<NA>,<NA>,<NA>,<NA>,10,1196,82,82,91,72,19,163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89528,97,137,<NA>,<NA>,<NA>,<NA>,20,2358,117,117,137,97,40,234
89529,137,<NA>,<NA>,<NA>,<NA>,<NA>,0,4692,137,137,137,137,0,137
89530,156,165,<NA>,<NA>,<NA>,<NA>,4,5456,160,160,165,156,9,321
89531,119,130,<NA>,<NA>,<NA>,<NA>,6,3141,124,124,130,119,11,249


In [785]:
df_gamestats_split = pd.concat([df_gamestats, df_games_split], axis=1)
df_gamestats_split = df_gamestats_split.drop('pins', axis=1)
df_gamestats_split


,player_name,player_id,gender,grade_level,school,match_id,location,opponent,num,game1,...,game5,game6,std,var,med,avg,max,min,rng,tot
0,Jennifer Calai,505715,Girls,Junior,Mt. Healthy,130115,Oxford Lanes,Edgewood,1,68,...,<NA>,<NA>,0,1156,68,68,68,68,0,68
1,Jennifer Calai,505715,Girls,Junior,Mt. Healthy,132603,Eastgate Lanes,Little Miami,2,90,...,<NA>,<NA>,7,1833,97,97,104,90,14,194
2,Diamond Johnson,314151,Girls,Senior,Withrow,78080,Stones Lanes,Turpin,1,86,...,<NA>,<NA>,0,1849,86,86,86,86,0,86
3,Diamond Johnson,314151,Girls,Senior,Withrow,78575,Stone Lanes,Winton Woods,2,111,...,<NA>,<NA>,5,2284,106,106,111,101,10,212
4,Diamond Johnson,314151,Girls,Senior,Withrow,78576,Stone Lanes,Loveland,2,91,...,<NA>,<NA>,10,1196,82,82,91,72,19,163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89528,Jaria Gray,96381,Girls,Senior,Chaminade Julienne,30458,Poelking Lanes,Fenwick,2,97,...,<NA>,<NA>,20,2358,117,117,137,97,40,234
89529,Jaria Gray,96381,Girls,Senior,Chaminade Julienne,30462,Poelking Lanes,Carroll,1,137,...,<NA>,<NA>,0,4692,137,137,137,137,0,137
89530,Jaria Gray,96381,Girls,Senior,Chaminade Julienne,30927,Poelking Lanes,Springboro,2,156,...,<NA>,<NA>,4,5456,160,160,165,156,9,321
89531,Jaria Gray,96381,Girls,Senior,Chaminade Julienne,34409,Poelking Lanes,Franklin,2,119,...,<NA>,<NA>,6,3141,124,124,130,119,11,249


In [786]:
df_gamestats_split.to_csv('gamestats_split.csv')


## Find school locations

Pulling locations from [OpenStreetMap](https://openstreetmap.org/copyright). Data is licensed under the [Open Data Commons Open Database License](https://opendatacommons.org/licenses/odbl/).

In [787]:
school_loc_rows = []
school_names = df_gamestats_split.school.unique()
for name in school_names:
    response = requests.get(f'https://nominatim.openstreetmap.org/search.php?q={name}+high,ohio&format=jsonv2', headers=headers).json()
    if len(response) >= 1:
        pass
    else:
        response = requests.get(f'https://nominatim.openstreetmap.org/search.php?q={name}+academy,ohio&format=jsonv2', headers=headers).json()
    # Some coordinates returned with ridiculous 15 decimal places; school locations don't need atomic-level precision, and even 5 is probably overkill
    lat = round(float(response[0]['lat']), 5)
    lon = round(float(response[0]['lon']), 5)
    display_name = response[0]['display_name']
    zip = re.search(pattern='(\d{5})', string=display_name).expand('\g<1>')
    school_loc_rows.append([name, lat, lon, display_name, zip])
    time.sleep(5)


In [788]:
df_school_locations = pd.DataFrame(school_loc_rows, columns=['name', 'latitude', 'longitude', 'display_name', 'zip'])
df_school_locations.head()

,name,latitude,longitude,display_name,zip
0,Mt. Healthy,39.24105,-84.55412,"Mount Healthy Junior/Senior High School, Adams...",45231
1,Withrow,39.14227,-84.45159,"Withrow University High School, 2488, Madison ...",45208
2,Northwest,39.27020,-84.57778,"Northwest High School, Pippin Road, Mount Heal...",45231
3,Mason,39.35071,-84.30624,"William Mason High School, Lakeside Drive, Mas...",45040
4,Roger Bacon,39.16233,-84.50258,"Roger Bacon High School, Vine Street, Saint Be...",45217


In [789]:
# Franklin returned a rec center at a university for whatever reason; corrected this one manually for ease of use
df_school_locations.loc[40] = 'Franklin', 39.55327, -84.28708, 'Franklin High School, 750, East 4th Street, Franklin, Warren County, Ohio, 45005, United States', 45005
df_school_locations.head()

,name,latitude,longitude,display_name,zip
0,Mt. Healthy,39.24105,-84.55412,"Mount Healthy Junior/Senior High School, Adams...",45231
1,Withrow,39.14227,-84.45159,"Withrow University High School, 2488, Madison ...",45208
2,Northwest,39.27020,-84.57778,"Northwest High School, Pippin Road, Mount Heal...",45231
3,Mason,39.35071,-84.30624,"William Mason High School, Lakeside Drive, Mas...",45040
4,Roger Bacon,39.16233,-84.50258,"Roger Bacon High School, Vine Street, Saint Be...",45217


In [790]:
df_school_locations.to_csv('school_locations.csv')

# Find median income data

In [791]:
median_raw = requests.get('https://api.census.gov/data/2020/acs/acs5/subject?get=NAME,S1903_C03_001E&for=zip%20code%20tabulation%20area:*', headers=headers).content

In [824]:
# Request returned as bytes
df_median_income = pd.read_csv(BytesIO(median_raw))
df_median_income

,"[[""NAME""",S1903_C03_001E,zip code tabulation area],Unnamed: 3
0,"[""ZCTA5 29590""",30985,29590],NaN
1,"[""ZCTA5 93306""",54450,93306],NaN
2,"[""ZCTA5 93660""",39625,93660],NaN
3,"[""ZCTA5 93110""",93264,93110],NaN
4,"[""ZCTA5 93212""",42983,93212],NaN
...,...,...,...,...
33115,"[""ZCTA5 16623""",51667,16623],NaN
33116,"[""ZCTA5 16627""",45000,16627],NaN
33117,"[""ZCTA5 16634""",51500,16634],NaN
33118,"[""ZCTA5 16640""",55982,16640],NaN


In [825]:
# Fix messy conversion to DataFrame
df_median_income.drop(df_median_income.columns[[0, 3]], axis=1, inplace=True)
df_median_income.set_axis(['median_income', 'zip_code'], axis=1, inplace=True)
df_median_income['zip_code'] = df_median_income['zip_code'].apply(lambda x: x.replace(']',''))
df_median_income = df_median_income.astype('int32')
# Remove codes outside of Ohio
df_median_income = df_median_income.loc[df_median_income['zip_code'].between(42999, 46000)]
df_median_income

,median_income,zip_code
13,38542,45237
14,66504,45788
15,50417,45819
146,66326,43212
147,76831,43214
...,...,...
29221,52403,43613
29222,47939,43615
29223,35030,43716
29224,41373,43723


In [794]:
df_median_income.to_csv('median_income_by_zip.csv')

# Find bowling alley locations

In [795]:
page = requests.get('https://www.kidsbowlfree.com/all_centers.php', headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')

In [796]:
alley_as = soup.find_all('a', {'title': re.compile('OH|KY|IN')})

In [797]:
alley_ids = []
for i in range(len(alley_as)):
    match = re.findall(r'id=(\d*)', str(alley_as[i]))[0]
    alley_ids.append(match)

In [798]:
alley_rows_list = []
for i in range(len(alley_ids)):
    page = requests.get(f'https://www.kidsbowlfree.com/center.php?alley_id={alley_ids[i]}', headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    name = soup.find_all('h1')[0].text.rstrip()
    address = soup.find_all('h4')[0].text.split('\n')
    street = address[0]
    rest = address[1].split(',')
    city = rest[0].lstrip()
    state = rest[1].lstrip()
    zip = rest[2].lstrip()[:5]
    try:
        response = requests.get(f'https://nominatim.openstreetmap.org/search.php?q={street}+{state}&format=jsonv2', headers=headers).json()
        if len(response) >=1:
            lat = round(float(response[0]['lat']), 5)
            lon = round(float(response[0]['lon']), 5)
        else:
            response = requests.get(f'https://nominatim.openstreetmap.org/search.php?q={name}+{state}&format=jsonv2', headers=headers).json()
            if len(response) >=1:
                lat = round(float(response[0]['lat']), 5)
                lon = round(float(response[0]['lon']), 5)
            else:
                lat = 0
                lon = 0
    except ValueError:  
        print(f'Decoding JSON failed for {name} {street} {state}')
    row =[name, street, city, state, zip, lat, lon]
    alley_rows_list.append(row)
    time.sleep(15)

Decoding JSON failed for Stones Lanes 3746 Montgomery Road #2 OH


In [799]:
df_alleys = pd.DataFrame(alley_rows_list, columns=['name', 'street_address', 'city', 'state', 'zip', 'latitude', 'longitude'])
df_alleys.loc[df_alleys['name'] == 'Stones Lanes', ['latitude', 'longitude']] = 39.14765, -84.46630
df_alleys

,name,street_address,city,state,zip,latitude,longitude
0,Beech Grove Bowl,95 N 2nd Ave,Beech Grove,IN,46107,37.97834,-87.57610
1,Classic Bowling Lanes,1421 N Willis Dr,Bloomington,IN,47404,39.18003,-86.54533
2,IMU Bowling & Billiards,900 E. Seventh St.,Bloomington,IN,47405,39.22844,-87.04407
3,Blackiston Bowl,1516 Blackiston Mill Rd,Clarksville,IN,47129,38.31385,-85.77146
4,Clarksville Strike & Spare,900 Eastern Blvd,Clarksville,IN,47129,38.30337,-85.76479
...,...,...,...,...,...,...,...
112,Le Ella Lanes,1428 US 22 NW,Washington Court House,OH,43160,0.00000,0.00000
113,Le Ella Lanes of Wellston,15 N Park Ave,Wellston,OH,45692,40.92201,-81.10414
114,Dynasty Lanes,3105 S.R. 103 East,Willard,OH,44890,0.00000,0.00000
115,Royal Z Lanes,2667 St Rt 22&3,Wilmington,OH,45177,48.03349,-71.29568


In [ ]:
# df_alleys.to_csv('alley_addresses.csv')

In [800]:
df_alleys = df_alleys.loc[((df_alleys['latitude'].between(38.75, 40.5))&(df_alleys['longitude'].between(-85.5, -83.75))),:]
df_alleys = df_alleys.drop_duplicates().reset_index(drop=True)
df_alleys

,name,street_address,city,state,zip,latitude,longitude
0,Hi-Way Lanes,400 N. Morton,Franklin,IN,46131,39.83722,-84.89520
1,Richmond 40 Bowl,75 S 37th St,Richmond,IN,47374,39.82520,-84.85065
2,Southern Lanes,7634 Alexandria Pike,Alexandria,KY,41001,38.97744,-84.39221
3,Strike & Spare Erlanger,510 Commonwealth Ave,Erlanger,KY,41018,39.02066,-84.60484
4,La Ru Bowling Lanes,2443 Alexandria Pike,Highland Heights,KY,41076,39.03854,-84.45038
5,Luray Lanes,647 E Main St,Ashland,OH,44805,39.92387,-83.79960
6,Spevock's Nautical Lanes,184 Miller Road,Avon Lake,OH,44012,39.45768,-84.20912
7,Batavia Bowl,1991 James E. Sauls Drive,Batavia,OH,45103,39.07331,-84.11340
8,Al-Mar Lanes,1010 North Main Street,Bowling Green,OH,43402,39.77380,-84.19848
9,Poelking Lanes South (Dayton Mall),8871 Kingsridge,Centerville,OH,45458,39.63059,-84.21759


In [801]:
df_alleys.to_csv('alley_addresses_bounded.csv')

In [802]:
alley_list = df_alleys.values.tolist()

In [803]:
df_dist = df_school_locations.copy()

In [804]:
for i in range(len(alley_list)):
    x1 = df_dist['longitude'].apply(radians)
    y1 = df_dist['latitude'].apply(radians)
    x2 = radians(alley_list[i][6])
    y2 = radians(alley_list[i][5])
    x = ((x1 - x2) * np.cos(0.5 * (y1 + y2)))
    y = (y1 - y2)
    df_dist[f'{alley_list[i][0]}'] = round((3960 * np.sqrt((x*x) + (y*y))), 2)

In [805]:
df_dist['min_dist'] = df_dist.iloc[:, 5:].min(axis=1)

In [806]:
df_dist['under_1'] = df_dist.iloc[:, 5:].apply(lambda x: (x<1).sum(), axis=1)
df_dist['under_5'] = df_dist.iloc[:, 5:].apply(lambda x: (x<5).sum(), axis=1)
df_dist['under_10'] = df_dist.iloc[:, 5:].apply(lambda x: (x<10).sum(), axis=1)

In [807]:
df_dist.to_csv('school_alley_distance.csv')

# Sqlite database creation

In [808]:
conn = db.connect('hs_bowl_sw_district_oh.db')

In [809]:
cur = conn.cursor()

In [810]:
df_alleys.to_sql('alleys', conn, if_exists='replace')

23

In [811]:
df_baker_gamestats_split.to_sql('baker_games', conn, if_exists='replace')

14286

In [812]:
df_gamestats_split.to_sql('games', conn, if_exists='replace')

89533

In [813]:
df_median_income.to_sql('median_income', conn, if_exists='replace')

1660

In [814]:
df_dist.to_sql('schools', conn, if_exists='replace')

59

In [815]:
df_matches_totals.to_sql('matches', conn, if_exists='replace')

11033

In [816]:
conn.commit()

In [817]:
cur.close()
conn.close()